In [1]:
from geopy.distance import geodesic
import numpy as np

In [ ]:
vl_madalena = (-23.553039, -46.686074)
urubici = (-27.999667, -49.586892)
mamangua = (-23.233871, -44.613234)

In [ ]:
geodesic(vl_madalena, urubici, ellipsoid='WGS-84').km #573.37 km

In [ ]:
geodesic(vl_madalena, mamangua, ellipsoid='WGS-84').km #214.49 km

In [ ]:
geodesic(mamangua, urubici, ellipsoid='WGS-84').km #727.50 km

In [2]:
from typing import Dict, Tuple


def geodistance(nodeA: str, nodeB: str, nodes_coords: Dict[str, Tuple]) -> float:
    nodeA_coords = nodes_coords[nodeA]
    nodeB_coords = nodes_coords[nodeB]
    distance = geodesic(nodeA_coords, nodeB_coords, ellipsoid='WGS-84').km 
    return distance


def geodistance_from_pair(nodes_pair: Tuple[str], nodes_coord: Dict[str, Tuple]) -> float:
    distance = geodistance(nodes_pair[0], nodes_pair[1], nodes_coord)
    return distance

In [ ]:
geodistance(nodeA='DE111', nodeB='DEF0C', nodes_coords=nodes_coords)

In [ ]:
geodistance_from_pair(('DE111', 'DEF0C'), nodes_coords)

# Getting Specific

In [3]:
import pandas as pd

In [4]:
centroids = pd.read_hdf('../data/04_feature/power-centroids-positions-2000-2015.hdf')

In [5]:
targets = ['DEF07', 'DEF0C', 'DEF05', 'DEF0E', 'DEF0B']

In [6]:
centroids_median = centroids.loc['2000-01-01':'2015-12-31', targets].median()

centroids_median

nuts_id  coords
DEF07    lat       54.655317
         lon        8.974127
DEF0C    lat       54.631738
         lon        9.395936
DEF05    lat       54.110200
         lon        9.007904
DEF0E    lat       53.944684
         lon        9.480571
DEF0B    lat       54.296444
         lon        9.789379
dtype: float64

In [7]:
nodes_coords = {district_id: tuple( centroids_median[district_id] ) for district_id in targets}

nodes_coords

{'DEF07': (54.65531672517318, 8.974126972180605),
 'DEF0C': (54.63173774352925, 9.39593596801835),
 'DEF05': (54.110200194884904, 9.007904152162004),
 'DEF0E': (53.94468431881774, 9.480570615203577),
 'DEF0B': (54.29644448857303, 9.789379082515651)}

In [8]:
from itertools import combinations

In [9]:
node_pairs = list( combinations(targets, 2) )

node_pairs

[('DEF07', 'DEF0C'),
 ('DEF07', 'DEF05'),
 ('DEF07', 'DEF0E'),
 ('DEF07', 'DEF0B'),
 ('DEF0C', 'DEF05'),
 ('DEF0C', 'DEF0E'),
 ('DEF0C', 'DEF0B'),
 ('DEF05', 'DEF0E'),
 ('DEF05', 'DEF0B'),
 ('DEF0E', 'DEF0B')]

In [10]:
nodes_pairwise_distances = {pair: geodistance_from_pair(pair, nodes_coords) for pair in node_pairs}

nodes_pairwise_distances

{('DEF07', 'DEF0C'): 27.358242338513392,
 ('DEF07', 'DEF05'): 60.71774710099849,
 ('DEF07', 'DEF0E'): 85.69686649210546,
 ('DEF07', 'DEF0B'): 66.24758261298138,
 ('DEF0C', 'DEF05'): 63.29408872529291,
 ('DEF0C', 'DEF0E'): 76.67449850720566,
 ('DEF0C', 'DEF0B'): 45.20905335272591,
 ('DEF05', 'DEF0E'): 36.03953640436726,
 ('DEF05', 'DEF0B'): 55.048036841626626,
 ('DEF0E', 'DEF0B'): 44.05329081328752}

In [11]:
distances_mx = pd.DataFrame(
    columns=targets,
    index=targets,
    data=None,
)

distances_mx

,DEF07,DEF0C,DEF05,DEF0E,DEF0B
DEF07,NaN,NaN,NaN,NaN,NaN
DEF0C,NaN,NaN,NaN,NaN,NaN
DEF05,NaN,NaN,NaN,NaN,NaN
DEF0E,NaN,NaN,NaN,NaN,NaN
DEF0B,NaN,NaN,NaN,NaN,NaN


In [ ]:
A = pd.DataFrame(
    columns=targets,
    index=targets,
    data=None,
)

In [ ]:
np.exp (D)

In [ ]:
D.values

In [ ]:
mya = np.array([[0,0,1], [2, 1, 3]])
mya

In [ ]:
np.exp( mya )

In [ ]:
np.exp( -np.square(D / std) )

In [ ]:
np.exp(-D.values/std)

In [ ]:
std = D.values.std()
A = np.exp(-np.square(D / std))

A

In [12]:
def build_distances_mx(node_pairs: Tuple[str], nodes_coords: Dict[str, Tuple[float]]) -> pd.DataFrame:
    # initialize 
    distances_mx = pd.DataFrame(
        columns=targets,
        index=targets,
        data=np.nan,
    )
    
    # calculate pairwise distances for upper triangle
    for pair in node_pairs:
        nodeA, nodeB = pair[0], pair[1]
        distances_mx.loc[nodeA, nodeB] = geodistance_from_pair(pair, nodes_coords)
    
    # mirror upper into lower triangle
    distances_mx.update(distances_mx.T)  # distance B-A = distance A-B 
    
    # fill diagonal with zeroes
    np.fill_diagonal( distances_mx.values, 0.0 )  # distance A-A = 0.0
    
    return distances_mx

In [13]:
def build_adjacency_mx(dist_dataframe: pd.DataFrame) -> pd.DataFrame:
    D = dist_dataframe
    
    std = D.values.std()
    A = np.exp( -np.square(D / std) )
    return A

In [61]:
D = build_distances_mx(node_pairs, nodes_coords)

D

,DEF07,DEF0C,DEF05,DEF0E,DEF0B
DEF07,0.000000,27.358242,60.717747,85.696866,66.247583
DEF0C,27.358242,0.000000,63.294089,76.674499,45.209053
DEF05,60.717747,63.294089,0.000000,36.039536,55.048037
DEF0E,85.696866,76.674499,36.039536,0.000000,44.053291
DEF0B,66.247583,45.209053,55.048037,44.053291,0.000000


In [76]:
A = build_adjacency_mx(D)

A

,DEF07,DEF0C,DEF05,DEF0E,DEF0B
DEF07,1.000000,0.363621,0.006854,0.000049,0.002654
DEF0C,0.363621,1.000000,0.004451,0.000354,0.063134
DEF05,0.006854,0.004451,1.000000,0.172815,0.016643
DEF0E,0.000049,0.000354,0.172815,1.000000,0.072581
DEF0B,0.002654,0.063134,0.016643,0.072581,1.000000


In [75]:
A_tril = pd.DataFrame(
    data= np.tril(A) - np.identity(len(A)), # exclusive lower triangle matrix of A 
    columns=A.columns,
    index=A.index
)

A_tril

,DEF07,DEF0C,DEF05,DEF0E,DEF0B
DEF07,0.000000,0.000000,0.000000,0.000000,0.0
DEF0C,0.363621,0.000000,0.000000,0.000000,0.0
DEF05,0.006854,0.004451,0.000000,0.000000,0.0
DEF0E,0.000000,0.000354,0.172815,0.000000,0.0
DEF0B,0.002654,0.063134,0.016643,0.072581,0.0


In [77]:
A_sparse = A.copy(deep=True)
A_sparse[D > 85] = 0

A_sparse

,DEF07,DEF0C,DEF05,DEF0E,DEF0B
DEF07,1.000000,0.363621,0.006854,0.000000,0.002654
DEF0C,0.363621,1.000000,0.004451,0.000354,0.063134
DEF05,0.006854,0.004451,1.000000,0.172815,0.016643
DEF0E,0.000000,0.000354,0.172815,1.000000,0.072581
DEF0B,0.002654,0.063134,0.016643,0.072581,1.000000


In [79]:
import pickle
pickle.dump(A, open('../data/05_model_input/adj_mx.pkl', 'wb'))